17102022

Бруцкий-Стемпковский

v. 01


Подготовить данные для этапа моделирования (обучения)
    
1. Анализ данных, необходимые корректировки. 
2. Обработать пропуски. 
3. Оценить выбросы. 
4. Корреляция. 
5. Тест на нормальность распределения.
6. Масштабировать и стандартизировать данные.
7. Разделите данные на тренировачную и тестовую выборку.
8. При обработке пропусков/выбросов/скалировании тестовую и тренировачную выборку обрабатывать отдельно.

Необходимые импорты библиотек.

In [52]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

Прочитаем фрейм. Переименуем колонки.

In [53]:
columns = [ 'id', 'gender', 'age', 
            'marital_status', 'job', 
            'sum_of_credit', 'credit_term',
            'tarrif_id', 'score_shk', 'edu', 
            'region', 'income', 'credit_count', 
            'overdue_credit_count', 'flag'
            ]
data = pd.read_csv('credit_train.csv', names=columns, skiprows=1, encoding='utf8', sep=';', thousands=",")
data.set_index("id", inplace=True)
data.head()

,gender,age,marital_status,job,sum_of_credit,credit_term,tarrif_id,score_shk,edu,region,income,credit_count,overdue_credit_count,flag
id,,,,,,,,,,,,,,
1,M,NaN,NaN,UMN,5999800.0,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
2,F,NaN,MAR,UMN,1088900.0,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
3,M,32.0,MAR,SPC,1072800.0,12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
4,F,27.0,NaN,SPC,1200909.0,12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
5,M,45.0,NaN,SPC,NaN,10,1.1,421385.0,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


Проверим информацию о строках.

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170746 entries, 1 to 170746
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job                   170746 non-null  object 
 4   sum_of_credit         170744 non-null  float64
 5   credit_term           170746 non-null  int64  
 6   tarrif_id             170746 non-null  float64
 7   score_shk             170739 non-null  float64
 8   edu                   170741 non-null  object 
 9   region                170554 non-null  object 
 10  income                170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  flag                  170746 non-null  int64  
dtypes: float64(7), int64(2), object(5)
memory usage: 19.

Фрейм содержит 14 признаков. Как числовые, так и категориальные признаки. Целевой показатель (функция от всех остальных аргументов) зашит в последней колонке 'flag'.

Большое количество выбросов сдержится в категориях о количестве имеющихся непогашенных и просроченных кредитов. От выбросов в остальных колонка просто избавимся.

Можно было бы заменить значения отдельных столбцов на среднее для числовых значений (применим это для столбиков с кредитами), либо заменить на самое часто встречающееся для категориальных значений.

In [55]:
drop_columns = [ 
            'gender', 'age', 'marital_status', 'job', 'sum_of_credit', 'credit_term',
            'tarrif_id', 'score_shk', 'edu', 'region', 'income', 'flag'
            ]
data.dropna(subset=drop_columns, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170543 entries, 8 to 170746
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170543 non-null  object 
 1   age                   170543 non-null  float64
 2   marital_status        170543 non-null  object 
 3   job                   170543 non-null  object 
 4   sum_of_credit         170543 non-null  float64
 5   credit_term           170543 non-null  int64  
 6   tarrif_id             170543 non-null  float64
 7   score_shk             170543 non-null  float64
 8   edu                   170543 non-null  object 
 9   region                170543 non-null  object 
 10  income                170543 non-null  float64
 11  credit_count          161331 non-null  float64
 12  overdue_credit_count  161331 non-null  float64
 13  flag                  170543 non-null  int64  
dtypes: float64(7), int64(2), object(5)
memory usage: 19.

In [56]:
data.head()

,gender,age,marital_status,job,sum_of_credit,credit_term,tarrif_id,score_shk,edu,region,income,credit_count,overdue_credit_count,flag
id,,,,,,,,,,,,,,
8,F,26.0,UNM,SPC,4787800.0,10,1.1,512525.0,GRD,МОСКОВСКАЯ ОБЛ,60000.0,3.0,0.0,0
10,F,32.0,UNM,SPC,2626800.0,10,1.1,465026.0,GRD,КРАЙ СТАВРОПОЛЬСКИЙ,39500.0,7.0,0.0,0
11,M,26.0,MAR,SPC,1679300.0,14,1.0,445430.0,SCH,САНКТ-ПЕТЕРБУРГ,36000.0,2.0,0.0,0
13,M,37.0,MAR,SPC,4228900.0,10,1.6,691609.0,SCH,ОБЛ АРХАНГЕЛЬСКАЯ,70000.0,1.0,0.0,0
14,M,43.0,MAR,INP,5756700.0,10,1.1,341164.0,GRD,ХАНТЫ-МАНСИЙСКИЙ АО,60000.0,7.0,0.0,0


С помощью метода .unique() исследуем отдельные колонки на нехарактерные значения.

In [57]:
"""file_obj = open("uniq_values.txt", "w")
columns = [ 'gender', 'age', 
            'marital_status', 'job', 
            'credit_term', 'tarrif_id', 'edu', 
            'region', 'credit_count', 
            'overdue_credit_count', 'flag'
            ]
for index in columns:
    file_obj.write(f"\n{index}\n\n")
    [file_obj.write(str(data[index].unique()[_]) + "\n") for _ in range(len(data[index].unique()))]
file_obj.close()""";

По результатам анализа видно следующее:

1. Колонки 'gender', 'age', 'marital_status', 'job', 'credit_term', 'tarrif_id', 'edu', 'income', 'flag'
очищены от испорченных данных. То есть их тип соотвествует ожидаемому, нет выбросов.

2. Колонки 'sum_of_credit' и 'score_shk' имеют большое число пропусков, но заменять пропуски мы будем после разбивки на тренировочоный и тестовый сеты.

3. Колонка 'region' имеет 301 уникальное значение, представляющее собой как строковые записи (повторяются по своей сути, но не по значению), встречаются числовые значения (74 и 98), вероятно представляющие собой номера регионов (Челябинск и Санкт-Петербург соответсвенно).

В этой версии мы избавимся от колонки "region".

In [58]:
data.drop(["region"], axis=1, inplace=True)

Разделим сет на тренировочную и тестовую выборки в отношении 70/30 соответсвенно. Деление выполним с помощью библиотеки sklearn. Указанные параметры указывают на то, что размер тестовой выборки будет составлять 30 %, а повторное деление сета даст тот же результат, что и при первом запуске. Далее показатель random_state можно удалить и тогда деление будет осуществляться каим-то новым образом.

In [59]:
train, test = train_test_split(data, test_size=0.3, random_state=2)

In [60]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119380 entries, 8155 to 89368
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                119380 non-null  object 
 1   age                   119380 non-null  float64
 2   marital_status        119380 non-null  object 
 3   job                   119380 non-null  object 
 4   sum_of_credit         119380 non-null  float64
 5   credit_term           119380 non-null  int64  
 6   tarrif_id             119380 non-null  float64
 7   score_shk             119380 non-null  float64
 8   edu                   119380 non-null  object 
 9   income                119380 non-null  float64
 10  credit_count          112895 non-null  float64
 11  overdue_credit_count  112895 non-null  float64
 12  flag                  119380 non-null  int64  
dtypes: float64(7), int64(2), object(4)
memory usage: 12.8+ MB


In [61]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51163 entries, 1019 to 40557
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gender                51163 non-null  object 
 1   age                   51163 non-null  float64
 2   marital_status        51163 non-null  object 
 3   job                   51163 non-null  object 
 4   sum_of_credit         51163 non-null  float64
 5   credit_term           51163 non-null  int64  
 6   tarrif_id             51163 non-null  float64
 7   score_shk             51163 non-null  float64
 8   edu                   51163 non-null  object 
 9   income                51163 non-null  float64
 10  credit_count          48436 non-null  float64
 11  overdue_credit_count  48436 non-null  float64
 12  flag                  51163 non-null  int64  
dtypes: float64(7), int64(2), object(4)
memory usage: 5.5+ MB


Теперь заполним пропуски для обеих выборок в колонках о количестве кредитов, заменив их на среднее значение.

In [62]:
train["credit_count"].fillna(train["credit_count"].mean(), inplace=True)
train["overdue_credit_count"].fillna(train["overdue_credit_count"].mean(), inplace=True)

test["credit_count"].fillna(test["credit_count"].mean(), inplace=True)
test["overdue_credit_count"].fillna(test["overdue_credit_count"].mean(), inplace=True)

Запишем получившиеся последовательности в csv-файлы.

In [63]:
train.to_csv("./v1/train.csv")
test.to_csv("./v1/test.csv")

Применим станартизацию к отдельным колонкам фрейма.

In [64]:
st_columns = ["age", "sum_of_credit", "credit_term", "income", "credit_count", "overdue_credit_count"]
no_st_columns = ['gender', 'marital_status', 'job', 'tarrif_id', 'score_shk', 'edu', 'flag']

train_st_copy = train[st_columns].copy()
test_st_copy = test[st_columns].copy()

In [65]:
scaler = StandardScaler()

d = scaler.fit_transform(train_st_copy)
train_st_copy = pd.DataFrame(d, columns = st_columns)
train_st_copy =pd.concat([train_st_copy, train[no_st_columns]], axis=1)

d = scaler.fit_transform(test_st_copy)
test_st_copy = pd.DataFrame(d, columns = st_columns)
test_st_copy =pd.concat([test_st_copy, test[no_st_columns]], axis=1)

In [66]:
train_st_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155274 entries, 0 to 162282
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   age                   119380 non-null  float64
 1   sum_of_credit         119380 non-null  float64
 2   credit_term           119380 non-null  float64
 3   income                119380 non-null  float64
 4   credit_count          119380 non-null  float64
 5   overdue_credit_count  119380 non-null  float64
 6   gender                119380 non-null  object 
 7   marital_status        119380 non-null  object 
 8   job                   119380 non-null  object 
 9   tarrif_id             119380 non-null  float64
 10  score_shk             119380 non-null  float64
 11  edu                   119380 non-null  object 
 12  flag                  119380 non-null  float64
dtypes: float64(9), object(4)
memory usage: 16.6+ MB


In [67]:
test_st_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87044 entries, 0 to 141823
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   age                   51163 non-null  float64
 1   sum_of_credit         51163 non-null  float64
 2   credit_term           51163 non-null  float64
 3   income                51163 non-null  float64
 4   credit_count          51163 non-null  float64
 5   overdue_credit_count  51163 non-null  float64
 6   gender                51163 non-null  object 
 7   marital_status        51163 non-null  object 
 8   job                   51163 non-null  object 
 9   tarrif_id             51163 non-null  float64
 10  score_shk             51163 non-null  float64
 11  edu                   51163 non-null  object 
 12  flag                  51163 non-null  float64
dtypes: float64(9), object(4)
memory usage: 9.3+ MB


In [68]:
train.describe()

,age,sum_of_credit,credit_term,tarrif_id,score_shk,income,credit_count,overdue_credit_count,flag
count,119380.000000,1.193800e+05,119380.000000,119380.000000,1.193800e+05,119380.000000,119380.000000,119380.000000,119380.000000
mean,36.488222,2.610830e+06,10.974610,1.324572,4.692243e+05,40136.867524,2.108809,0.045786,0.175867
std,10.545115,1.624484e+06,3.527895,0.235919,1.244182e+05,24961.937325,1.726641,0.205070,0.380709
min,18.000000,3.000000e+05,3.000000,1.000000,0.000000e+00,6000.000000,0.000000,0.000000,0.000000
25%,28.000000,1.488375e+06,10.000000,1.100000,3.790640e+05,25000.000000,1.000000,0.000000,0.000000
50%,34.000000,2.125954e+06,10.000000,1.320000,4.612780e+05,35000.000000,2.000000,0.000000,0.000000
75%,43.000000,3.205985e+06,12.000000,1.600000,5.525800e+05,50000.000000,3.000000,0.000000,0.000000
max,71.000000,2.000000e+07,36.000000,1.940000,1.054699e+06,950000.000000,21.000000,3.000000,1.000000


In [69]:
test.describe()

,age,sum_of_credit,credit_term,tarrif_id,score_shk,income,credit_count,overdue_credit_count,flag
count,51163.000000,5.116300e+04,51163.000000,51163.000000,5.116300e+04,51163.000000,51163.000000,51163.000000,51163.000000
mean,36.526963,2.605954e+06,10.995876,1.322366,4.699636e+05,40137.110803,2.104571,0.046701,0.177101
std,10.551720,1.620607e+06,3.558966,0.235596,1.238728e+05,25244.358116,1.733547,0.207195,0.381758
min,18.000000,2.736000e+05,3.000000,1.000000,0.000000e+00,5000.000000,0.000000,0.000000,0.000000
25%,28.000000,1.495700e+06,10.000000,1.100000,3.804070e+05,25000.000000,1.000000,0.000000,0.000000
50%,34.000000,2.116700e+06,10.000000,1.320000,4.621600e+05,35000.000000,2.000000,0.000000,0.000000
75%,43.000000,3.207350e+06,12.000000,1.600000,5.519060e+05,50000.000000,3.000000,0.000000,0.000000
max,71.000000,2.000000e+07,36.000000,1.960000,1.128291e+06,900000.000000,18.000000,3.000000,1.000000


Приведенные выше таблицы есть своего рода оценки выбросов. Размах свыше трёх std имеется по существу во всех колонках. 

Для дальнейшего анализа кодируем категориальные значения в числовые с помощью метода get_dammies. Оставшиеся категориальные солбцы удаляем, вместо них появятся новые числовые столбцы.

In [70]:
train = pd.concat([ train, 
                    pd.get_dummies(train['gender'], prefix="gender"),
                    pd.get_dummies(train['marital_status'], prefix="marital_status"),
                    pd.get_dummies(train['job'], prefix="job"),
                    pd.get_dummies(train['edu'], prefix="edu")],
                    axis=1)
train.drop(["gender", "marital_status", "job", "edu"], axis=1, inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119380 entries, 8155 to 89368
Data columns (total 39 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   age                   119380 non-null  float64
 1   sum_of_credit         119380 non-null  float64
 2   credit_term           119380 non-null  int64  
 3   tarrif_id             119380 non-null  float64
 4   score_shk             119380 non-null  float64
 5   income                119380 non-null  float64
 6   credit_count          119380 non-null  float64
 7   overdue_credit_count  119380 non-null  float64
 8   flag                  119380 non-null  int64  
 9   gender_F              119380 non-null  uint8  
 10  gender_M              119380 non-null  uint8  
 11  marital_status_CIV    119380 non-null  uint8  
 12  marital_status_DIV    119380 non-null  uint8  
 13  marital_status_MAR    119380 non-null  uint8  
 14  marital_status_UNM    119380 non-null  uint8  
 15

In [71]:
test = pd.concat([  test, 
                    pd.get_dummies(test['gender'], prefix="gender"),
                    pd.get_dummies(test['marital_status'], prefix="marital_status"),
                    pd.get_dummies(test['job'], prefix="job"),
                    pd.get_dummies(test['edu'], prefix="edu")],
                    axis=1)
test.drop(["gender", "marital_status", "job", "edu"], axis=1, inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51163 entries, 1019 to 40557
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   age                   51163 non-null  float64
 1   sum_of_credit         51163 non-null  float64
 2   credit_term           51163 non-null  int64  
 3   tarrif_id             51163 non-null  float64
 4   score_shk             51163 non-null  float64
 5   income                51163 non-null  float64
 6   credit_count          51163 non-null  float64
 7   overdue_credit_count  51163 non-null  float64
 8   flag                  51163 non-null  int64  
 9   gender_F              51163 non-null  uint8  
 10  gender_M              51163 non-null  uint8  
 11  marital_status_CIV    51163 non-null  uint8  
 12  marital_status_DIV    51163 non-null  uint8  
 13  marital_status_MAR    51163 non-null  uint8  
 14  marital_status_UNM    51163 non-null  uint8  
 15  marital_status_W

Тест Шапиро-Уилка

In [72]:
def normal_test(ser):
    stat, p = sp.stats.shapiro(ser)

    alpha = 0.05
    if p > alpha:
        return 'Похоже на нормальное распределение'
    else:
        return 'Не похоже на нормальное распределение'


In [73]:
for name in train.columns[:9]:
    print(name, normal_test(train[name]))

age Не похоже на нормальное распределение
sum_of_credit Не похоже на нормальное распределение
credit_term Не похоже на нормальное распределение
tarrif_id Не похоже на нормальное распределение
score_shk Не похоже на нормальное распределение
income Не похоже на нормальное распределение
credit_count Не похоже на нормальное распределение
overdue_credit_count Не похоже на нормальное распределение
flag Не похоже на нормальное распределение


c:\Users\Николай\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [74]:
for name in test.columns[:9]:
    print(name, normal_test(test[name]))

age Не похоже на нормальное распределение
sum_of_credit Не похоже на нормальное распределение
credit_term Не похоже на нормальное распределение
tarrif_id Не похоже на нормальное распределение
score_shk Не похоже на нормальное распределение
income Не похоже на нормальное распределение
credit_count Не похоже на нормальное распределение
overdue_credit_count Не похоже на нормальное распределение
flag Не похоже на нормальное распределение


In [75]:
corr_matrix_train = train.corr()
corr_matrix_test = test.corr()

In [76]:
corr_matrix_train

,age,sum_of_credit,credit_term,tarrif_id,score_shk,income,credit_count,overdue_credit_count,flag,gender_F,...,job_SPC,job_UMN,job_WOI,job_WRK,job_WRP,edu_ACD,edu_GRD,edu_PGR,edu_SCH,edu_UGR
age,1.000000,-0.098405,0.006240,-0.110119,-0.169003,-0.015153,1.143208e-01,7.081771e-03,-0.034748,0.125798,...,-0.155729,-0.004007,-0.002472,-0.008503,0.049079,0.002233,-0.060217,-0.003911,0.119265,-0.126086
sum_of_credit,-0.098405,1.000000,0.229523,0.091613,0.055526,0.348630,1.640763e-02,-2.993325e-03,-0.073952,-0.033438,...,-0.072479,0.067233,-0.004422,-0.013467,-0.005231,0.011328,0.176898,0.019019,-0.186291,0.018282
credit_term,0.006240,0.229523,1.000000,-0.058819,0.042529,0.007924,4.825116e-02,4.581232e-03,0.025228,0.001231,...,-0.003917,-0.010776,0.006244,-0.002002,0.005786,-0.000760,0.007421,0.011882,-0.006157,-0.005321
tarrif_id,-0.110119,0.091613,-0.058819,1.000000,0.397488,0.015002,-5.122880e-02,8.231539e-02,-0.066149,-0.012238,...,0.025744,-0.021485,0.003426,0.010538,-0.000447,0.007633,-0.061665,0.003974,0.037716,0.047605
score_shk,-0.169003,0.055526,0.042529,0.397488,1.000000,-0.104909,-9.110441e-02,1.622842e-01,0.050244,0.011650,...,0.038415,-0.046544,0.005361,0.013789,-0.001096,-0.011410,-0.250092,-0.034796,0.223641,0.059946
income,-0.015153,0.348630,0.007924,0.015002,-0.104909,1.000000,1.431234e-01,3.544061e-02,-0.025408,-0.202955,...,-0.155875,0.126024,-0.007033,-0.006562,0.000714,0.022948,0.215566,0.049388,-0.229871,0.020980
credit_count,0.114321,0.016408,0.048251,-0.051229,-0.091104,0.143123,1.000000e+00,1.753623e-01,0.033353,0.013617,...,-0.025433,0.024947,0.000283,0.008099,0.021396,0.005375,0.031541,0.013758,-0.026002,-0.014943
overdue_credit_count,0.007082,-0.002993,0.004581,0.082315,0.162284,0.035441,1.753623e-01,1.000000e+00,0.022867,-0.001975,...,-0.013156,0.010355,0.002100,0.007156,-0.000417,0.002610,0.022573,0.005414,-0.020767,-0.004899
flag,-0.034748,-0.073952,0.025228,-0.066149,0.050244,-0.025408,3.335251e-02,2.286680e-02,1.000000,-0.045536,...,-0.046619,0.011144,0.008121,-0.000706,0.004784,0.001425,-0.082885,0.001806,0.078661,0.006417
gender_F,0.125798,-0.033438,0.001231,-0.012238,0.011650,-0.202955,1.361690e-02,-1.975031e-03,-0.045536,1.000000,...,0.023421,-0.020893,0.000032,-0.021220,0.003115,-0.006327,0.074238,-0.008148,-0.062280,-0.020971


In [77]:
corr_matrix_test

,age,sum_of_credit,credit_term,tarrif_id,score_shk,income,credit_count,overdue_credit_count,flag,gender_F,...,job_SPC,job_UMN,job_WOI,job_WRK,job_WRP,edu_ACD,edu_GRD,edu_PGR,edu_SCH,edu_UGR
age,1.000000,-0.091942,0.009175,-0.106947,-0.159469,-0.014126,0.106003,0.008525,-0.022945,0.124726,...,-0.163095,0.004809,-0.009405,-0.013221,0.052694,0.004567,-0.061469,-0.007032,0.120473,-0.127710
sum_of_credit,-0.091942,1.000000,0.225852,0.088326,0.051578,0.357764,0.013031,-0.005744,-0.067636,-0.035318,...,-0.070325,0.066426,-0.005872,-0.013688,-0.003363,0.001643,0.183236,0.019206,-0.193520,0.021427
credit_term,0.009175,0.225852,1.000000,-0.050355,0.035472,0.007141,0.051200,0.010214,0.027080,-0.009041,...,-0.005156,-0.012415,0.007179,0.005524,0.015353,-0.015371,0.012959,0.017385,-0.011282,-0.006036
tarrif_id,-0.106947,0.088326,-0.050355,1.000000,0.399635,0.008145,-0.054674,0.088666,-0.070335,-0.014148,...,0.024604,-0.018927,0.004013,0.014821,-0.005969,-0.001023,-0.057630,0.002768,0.038952,0.038496
score_shk,-0.159469,0.051578,0.035472,0.399635,1.000000,-0.113130,-0.086349,0.164087,0.045496,0.014612,...,0.034650,-0.039523,0.001571,0.025909,-0.008991,-0.018770,-0.246943,-0.039905,0.223284,0.057512
income,-0.014126,0.357764,0.007141,0.008145,-0.113130,1.000000,0.146025,0.027859,-0.019715,-0.200498,...,-0.157522,0.126432,-0.008710,-0.010266,-0.005477,0.031909,0.211347,0.049247,-0.226431,0.021724
credit_count,0.106003,0.013031,0.051200,-0.054674,-0.086349,0.146025,1.000000,0.181779,0.035788,0.016258,...,-0.020418,0.029592,-0.003533,0.011990,-0.001231,0.006304,0.029122,0.016226,-0.023486,-0.016249
overdue_credit_count,0.008525,-0.005744,0.010214,0.088666,0.164087,0.027859,0.181779,1.000000,0.020829,-0.000021,...,-0.007118,0.013471,-0.001196,0.001258,-0.002682,-0.001716,0.018729,0.012151,-0.018180,-0.003613
flag,-0.022945,-0.067636,0.027080,-0.070335,0.045496,-0.019715,0.035788,0.020829,1.000000,-0.041688,...,-0.055149,0.012674,0.005886,0.005920,-0.003251,0.009383,-0.081443,0.003296,0.077824,0.004156
gender_F,0.124726,-0.035318,-0.009041,-0.014148,0.014612,-0.200498,0.016258,-0.000021,-0.041688,1.000000,...,0.023239,-0.026238,0.006909,-0.013154,0.004730,-0.014428,0.077864,-0.005203,-0.067296,-0.018249
